In [1]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns




In [2]:
url_test= "https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make=Ford&srchtyp=newMake&pageno=1&rowLimit=200&sortBy=Comb&tabView=0"
page_n1 = r.get(url_test)
soup = bs(page_n1.content)

In [3]:
page_n1

<Response [200]>

page = r.get(url)
soup = bs(page.content, "html.parser")

In [4]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

manufacturers = []
years = []
models = []
miles_IRA = []  # Initialize the list for total ranges

for maker in makers:
    page_number = 1
    while True:
        url = f"{base_url}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={maker}&srchtyp=newMake&pageno={page_number}&rowLimit=200&sortBy=Comb&tabView=0"
        page = r.get(url)
        soup_IRA = bs(page.content, "html.parser")
        rows = soup_IRA.find_all("tr", class_="ymm-row")
        
        if not rows:  # Check if there are no more rows, indicating the end of data
            break

        for tr in rows:
            a_tags = tr.find_all("a")
            for a in a_tags:
                manufacturers.append(a.text.split()[1])
                years.append(a.text.split()[0])
                models.append(' '.join(a.text.split()[2:]))   

        # Extract total ranges and append to the list
        miles_IRA.extend([mile_IRA.get_text() for mile_IRA in soup_IRA.find_all("div", class_="totalRange")])

        page_number += 1 


In [5]:
#trim column miles to that it matches the other columns ranges since I have no Nan values
miles_IRA = miles_IRA[:len(manufacturers)] 

In [6]:
data = {
    'Manufacturer': manufacturers,
    'Year': years,
    'Model': models,
    'Miles_IRA': miles_IRA
}

# Create a DataFrame from the dictionary
df_mateo = pd.DataFrame(data)

In [7]:
# checking my df
df_mateo

,Manufacturer,Year,Model,Miles_IRA
0,Chevrolet,2023,Bolt EV,259 milesTotal Range
1,Chevrolet,2022,Bolt EV,259 milesTotal Range
2,Chevrolet,2019,Bolt EV,238 milesTotal Range
3,Chevrolet,2021,Bolt EV,259 milesTotal Range
4,Chevrolet,2020,Bolt EV,259 milesTotal Range
...,...,...,...,...
1286,Nissan,2022,Armada 4WD,416 milesTotal Range
1287,Nissan,2021,Armada 4WD,416 milesTotal Range
1288,Nissan,2021,Armada 4WD,416 milesTotal Range
1289,Nissan,2020,Armada 4WD,390 milesTotal Range


In [8]:
#checking if my carray has same lenght
"""print("Length of manufacturers list:", len(manufacturers))
print("Length of years list:", len(years))
print("Length of models list:", len(models))
print("Length of miles_IRA list:", len(miles_IRA))"""


'print("Length of manufacturers list:", len(manufacturers))\nprint("Length of years list:", len(years))\nprint("Length of models list:", len(models))\nprint("Length of miles_IRA list:", len(miles_IRA))'

In [9]:
# checking for types in my df df_mateo.dtypes

In [10]:
# changing types from obj to int to be able to perform numr 
df_mateo['Year'] = df_mateo['Year'].astype(int)
df_mateo['Miles_IRA'] = df_mateo['Miles_IRA'].str.split().str[0].astype(int)

In [11]:
# Rename the 'Miles_IRA' column to 'Total_Range_in_MLS'
df_mateo = df_mateo.rename(columns={'Miles_IRA': 'Total_Range_in_MLS'})

# Convert column names to lower case and replace spaces with underscores
df_mateo.columns = df_mateo.columns.str.lower().str.replace(' ', '_')


In [12]:
df_mateo

,manufacturer,year,model,total_range_in_mls
0,Chevrolet,2023,Bolt EV,259
1,Chevrolet,2022,Bolt EV,259
2,Chevrolet,2019,Bolt EV,238
3,Chevrolet,2021,Bolt EV,259
4,Chevrolet,2020,Bolt EV,259
...,...,...,...,...
1286,Nissan,2022,Armada 4WD,416
1287,Nissan,2021,Armada 4WD,416
1288,Nissan,2021,Armada 4WD,416
1289,Nissan,2020,Armada 4WD,390
